## 객실의 사용 여부 관련 데이터 
1. 데이터를 로드 (hotel_bookings.csv)
2. 데이터에 대한 정보를 확인 
3. 해당 데이터에서 문제가 있는 부분을 확인하여 수정 

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
# data폴더 안에 hotel_bookings.csv 파일을 로드 

# 데이터를 로드 하고 문제점(랜덤포레스트 학습)

hotel = pd.read_csv("../data/hotel_bookings.csv")
hotel.head(2)

In [ ]:
# 데이터프레임의 정보 확인 
hotel.info()

### 해당 데이터의 컬럼의 의미 
is_canceled : 예약 취소 여부 (0=취소 안됨 ,1=취소 )

deposit_type : 보증금 유형 (No Deposit, Not Refund, Refundable)

lead_time : 예약일과 실제 도착일 사이의 일수 

stays_in_weekend_nights : 주말의 숙박 일수 

stays_in_week_nights : 주중의 숙박 일수

is_repeated_guest : 재방문 고객 여부 (0=신규, 1=재방문)

previous_cancellations : 과거 예약 취소 횟수

previous_bookings_not_canceled : 과거 예약 중 취소되지 않은 건수

booking_changes : 예약 후 변경 횟수

days_in_waiting_list : 대기자가 있었던 일수

adr : 평균 일일 객실 요금

In [ ]:
# 결측치가 존재하는것은 확인 -> 실제 결측치의 개수를 확인 
# 결측치가 존재 여부 함수 -> bool 타입의 데이터프레임이 생성 
# bool의 데이터들을 합산하여 컬럼별로 확인 
hotel.isna().sum()

In [ ]:
# 결측치의 비율 -> 결측치의 개수 / 데이터프레임의 길이 * 100
print("lead_time 컬럼의 결측치의 비율 ", round( 5 / len(hotel) * 100 , 2 ))
print('is_repeated_guest 컬럼의 결측치의 비율 ', 
      round( 358 / len(hotel) * 100, 2 ))
print('adr 컬럼의 결측치의 비율 ', round( 1063 / len(hotel) * 100, 2 ))

- lead_time 컬럼의 결측치의 비율은 매우 작기 때문에 제거 -> 결측치인 인덱스를 제외
- is_repeated_guest는 해당 데이터에서 개수가 많은 데이터로 결측치를 채워준다. 
- adr 컬럼의 결측치는 해당 데이터들을 확인하고 특정한 조건에 맞춰서 데이터를 채워준다. 

In [ ]:
# lead_time의 결측치가 존재하는 인덱스를 제외 
# 제거한다(drop) + 결측치(na) -> 결측치가 존재하는 행이나 열을 제거하는 함수 
hotel.dropna( subset=['lead_time'], axis = 0, inplace=True)

In [ ]:
# 값들의 빈도수를 체크하는 함수를 사용
hotel['is_repeated_guest'].value_counts()

In [ ]:
# is_repeated_guest 컬럼의 결측치는 0으로 채워준다. 
hotel['is_repeated_guest'].fillna(0, inplace=True)

In [ ]:
hotel.isna().sum()

In [17]:
hotel['adr'].describe()

count    18932.000000
mean       101.410702
std         49.241204
min         -6.380000
25%         68.822500
50%         94.500000
75%        126.000000
max        451.500000
Name: adr, dtype: float64

In [21]:
# 통계 정보를 확인하니 객실 요금 평균에 음수가 존재한다. -> 이상한 데이터가 발견 
# 이상치 데이터는 제거 
# 인덱스의 조건식을 생성 -> 객실 요금 데이터에서 0보다 작은 
flag = hotel['adr'] < 0

hotel = hotel.loc[~flag]

In [22]:
# adr의 결측치들을 deposit_type의 값에 따라 그룹화를 하고 평균의 adr의 값을 채워준다. 

# deposit_type에 따른 adr의 평균값을 확인
hotel.groupby(['deposit_type'])['adr'].mean()

deposit_type
No Deposit    101.971806
Non Refund     89.689476
Refundable     67.456154
Name: adr, dtype: float64

In [23]:
null_flag = hotel['adr'].isna()

hotel.loc[null_flag]

,is_canceled,deposit_type,lead_time,stays_in_weekend_nights,stays_in_week_nights,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr
2,0,No Deposit,33.0,2,3,0.0,0,0,0,0,NaN
11,0,No Deposit,77.0,2,2,0.0,0,0,0,0,NaN
14,0,No Deposit,377.0,0,2,0.0,0,0,1,0,NaN
26,0,No Deposit,0.0,0,1,0.0,0,0,0,0,NaN
31,0,No Deposit,67.0,2,4,0.0,0,0,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
19905,1,Non Refund,34.0,0,2,0.0,1,0,0,0,NaN
19910,1,No Deposit,49.0,0,4,0.0,0,0,0,0,NaN
19942,1,Non Refund,336.0,0,2,0.0,1,0,0,0,NaN
19953,1,No Deposit,292.0,0,3,0.0,0,0,0,0,NaN
